>Backpropagating errors to parameters and then updating those parameters by tak- ing the gradient with respect to the loss is the same no matter what the underlying model is.

The `torch.nn` module contains building blocks for building neural networks. 

The building blocks are called "modules" (analogous to "layers" in the literature). The base class is `nn.Module`.

A `nn.Module` can have `Parameter` instances as attributes; they can also have `nn.Module` instances as submodules. These must be top-level data attributes! Otherwise use `nn.ModuleList` or `nn.ModuleDict`.

One popular examples is `nn.Linear`, an affine transformation.

`nn.Module` classes all have `__call__` defined, so we can call them like a function. This computes a forward pass.

In [13]:
import torch

linear_model = torch.nn.Linear(in_features=1, out_features=1)

list(linear_model.named_parameters())

[('weight',
  Parameter containing:
  tensor([[0.0589]], requires_grad=True)),
 ('bias',
  Parameter containing:
  tensor([0.3777], requires_grad=True))]

Important! `nn.Module` is designed to take multiple samples at once. The zeroth dimension is the batch dim.

In [16]:
x = torch.ones(size=(10, 1))
x

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [17]:
linear_model(x)

tensor([[0.4366],
        [0.4366],
        [0.4366],
        [0.4366],
        [0.4366],
        [0.4366],
        [0.4366],
        [0.4366],
        [0.4366],
        [0.4366]], grad_fn=<AddmmBackward0>)

In [24]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_c_train, t_u_val = None, t_c_val = None):
    for epoch in range(1, n_epochs+1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        # t_p_val = model(t_u_val)
        # loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f}")
                #   f" Validation loss {loss_val.item():.4f}")

t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1)
t_u = torch.tensor(t_u).unsqueeze(1)
t_un = 0.1 * t_u

linear_model = torch.nn.Linear(1, 1)
optimizer = torch.optim.SGD(
    linear_model.parameters(),
    lr=1e-2,
)

training_loop(
    n_epochs=3000,
    model=linear_model,
    optimizer=optimizer,
    loss_fn=torch.nn.MSELoss(),
    t_u_train=t_un,
    t_c_train=t_c,
)

print("\n",linear_model.weight,"\n",linear_model.bias)

Epoch 1, Training loss 127.5501
Epoch 1000, Training loss 3.8621
Epoch 2000, Training loss 2.9588
Epoch 3000, Training loss 2.9287

 Parameter containing:
tensor([[5.3485]], requires_grad=True) 
 Parameter containing:
tensor([-17.1960], requires_grad=True)
